<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第2章　Step-1.-前処理" data-toc-modified-id="第2章　Step-1.-前処理-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第2章　Step 1. 前処理</a></span></li></ul></div>

# 第2章　Step 1. 前処理

in silico スクリーニングを行うためにsdfに含まれる化合物構造の前処理を行いました。

行った前処理は

1. 構造の標準化
2. 脱塩処理　（一番大きいサイズのフラグメントのみ残す）
3. 電荷の中和

です。また、あわせて次のステップで用いるための指標の計算を行いました。

1. 分子量
2. cLogP
3. 水素結合供与体数
4. 水素結合受容体数
5. 回転可能結合数
6. 極性表面積

RDKitをもちいて行った前処理コード例は以下のようなものとなります。

```python
from rdkit import Chem
import gzip
from rdkit.Chem import Descriptors, MolStandardize

EAc_gz = gzip.open('Enamine_Advanced_collection.sdf.gz')
EAc_suppl = Chem.ForwardSDMolSupplier(EAc_gz) 

#Molオブジェクト、および各指標を計算し格納するためのからのリストを作成
EAc_mols = []
MW_list = []
MolLogP_list = []
NumHDonors_list = []
NumHAcceptors_list = []
NumRotatableBonds_list = []
TPSA_list = []

for x in EAc_suppl:
  if x is not None:
    mol = x
    
  #処理の前にidnumberを取り出しておく
  ID = mol.GetProp('idnumber') 

    
  #構造の標準化
  normalizer =MolStandardize.normalize.Normalizer()
  mol_norm = normalizer.normalize(mol)
  
  #一番大きいサイズのフラグメントのみ残す
  lfc = MolStandardize.fragment.LargestFragmentChooser()
  mol_desalt = lfc.choose(mol_norm)
  
  #電荷の中和
  uc = MolStandardize.charge.Uncharger()
  mol_neu = uc.uncharge(mol_desalt)
  
  #処理後のMolオブジェクトに、取り出しておいた元々のidnumberをoriginal_idとして付与
  mol_neu.SetProp('original_id', ID)
  
  #分子量を計算
  MW_value = Chem.Descriptors.MolWt(mol_neu)

  #分子量をプロパティとして持たせる。(小数点を含むfloat型)
  mol_neu.SetDoubleProp('MW', MW_value)
  
  #値のみのリストにも追加する
  MW_list.append(MW_value)
  
  #MolLogPを計算(MolLogPは対応しない元素を含むとエラーを返すので例外処理として記入)
  try: 
    MolLogP_value = Chem.Descriptors.MolLogP(mol_neu)
    mol_neu.SetDoubleProp('MolLogP', MolLogP_value)
    MolLogP_list.append(MolLogP_value)
  except:
    MolLogP_list.append(None)
   
  #NumHDonorsでも同じことをする。
  NumHDonors_value = Chem.Descriptors.NumHDonors(mol_neu)
  #非負の整数値（int型）のプロパティなので SetUnsignedIntProp でも良いが面倒なのでSetDoublePropにする。
  mol_neu.SetDoubleProp('NumHDonors', NumHDonors_value)
  NumHDonors_list.append(NumHDonors_value)  

  #NumHAcceptors
  NumHAcceptors_value = Chem.Descriptors.NumHAcceptors(mol_neu)
  mol_neu.SetDoubleProp('NumHAcceptors', NumHAcceptors_value)
  NumHAcceptors_list.append(NumHAcceptors_value)  

  #NumRotatableBonds
  NumRotatableBonds_value = Chem.Descriptors.NumRotatableBonds(mol_neu)
  mol_neu.SetDoubleProp('NumRotatableBonds', NumRotatableBonds_value)
  NumRotatableBonds_list.append(NumRotatableBonds_value)    

  #TPSA
  TPSA_value = Chem.Descriptors.TPSA(mol_neu)
  mol_neu.SetDoubleProp('TPSA', TPSA_value)
  TPSA_list.append(TPSA_value)   
 
  #Molオブジェクトのリストも忘れずに
  EAc_mols.append(mol_neu)
```

上記コード例の詳細は**第３章３節**に記載していあります。

こちらに至るまでの過程として、第２章では

* 大きなデータセットを利用するためにGoogle colabolatoryを利用　（第２章１節）
* 化合物構造に紐づいたデータ取り扱い上の注意点（第２章３節）

を取り上げています。また、テクニカルな話題として

* 処理手順の変更による必要なメモリの節約　（第２章２節）
* SDF読みこみ方法の違いの影響　（第２章コラム）

について記載しています。